In [1]:
"""Generates annoy indexes and related function caches. Forked from explore/generateIndex.ipynb

TODO merge with explore/generateIndex.ipynb
"""


In [ ]:
from pathlib import Path

from nna.fileUtils import list_files,save_to_csv

import numpy as np

In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('/home/enis/projects/similarSoundsApp/'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
    



/home/enis/projects/similarSoundsApp


In [3]:
from nnSearch import loadOnDisk,buildOnDisk


#### Parameters

In [4]:
indexIDname = "photoEmbeddings_V3_norm_euc_custom"

outputFolder = Path("/scratch/enis/data/nna/realMerged/")
metadataCSV = outputFolder / (indexIDname+".csv")
fn = outputFolder / (indexIDname+".ann")

# vector size
f=128
  # Length of item vector that will be indexed
# distance='hamming'
distance='euclidean'

#prepares annoy to build the index in the specified file instead of RAM 
#(execute before adding items, no need to save after build)


### 1) Load embeddings to memory
#### 1.0) find images so we can just load embeddings from these locations


In [5]:
## Load datasets
realFolderPath="/scratch/enis/data/nna/real/"


# Look where the images are so we index only these locations
pathDatasets = "/home/enis/projects/similarSoundsApp/resources/TimeLapse_Databases/"
datasetList=[[m for m in i.glob("*") if ".ddb" in str(m) ][0] for i in (Path(pathDatasets).glob("*"))]
# locations with only images
labeledImgLocations = [m.parent.stem for m in datasetList]
# all existing locations
region_locId=[[m for m in i.glob("*") if m.is_dir() ] for i in (Path(realFolderPath).glob("*")) if i.is_dir()]

#keep locations only with images
allLocations=[]
for m in region_locId:
    for n in m:
        if n.stem in labeledImgLocations:
            allLocations.append(n)



In [6]:
allLocations

[PosixPath('/scratch/enis/data/nna/real/anwr/46'),
 PosixPath('/scratch/enis/data/nna/real/anwr/31'),
 PosixPath('/scratch/enis/data/nna/real/anwr/36'),
 PosixPath('/scratch/enis/data/nna/real/anwr/38'),
 PosixPath('/scratch/enis/data/nna/real/anwr/32'),
 PosixPath('/scratch/enis/data/nna/real/anwr/45'),
 PosixPath('/scratch/enis/data/nna/real/anwr/48'),
 PosixPath('/scratch/enis/data/nna/real/anwr/42'),
 PosixPath('/scratch/enis/data/nna/real/anwr/35'),
 PosixPath('/scratch/enis/data/nna/real/anwr/40'),
 PosixPath('/scratch/enis/data/nna/real/anwr/37'),
 PosixPath('/scratch/enis/data/nna/real/anwr/50'),
 PosixPath('/scratch/enis/data/nna/real/anwr/34'),
 PosixPath('/scratch/enis/data/nna/real/anwr/33'),
 PosixPath('/scratch/enis/data/nna/real/prudhoe/11'),
 PosixPath('/scratch/enis/data/nna/real/prudhoe/29'),
 PosixPath('/scratch/enis/data/nna/real/prudhoe/24'),
 PosixPath('/scratch/enis/data/nna/real/prudhoe/16'),
 PosixPath('/scratch/enis/data/nna/real/prudhoe/12'),
 PosixPath('/scr

#### 1.1 find embeddings in the given locations 

In [7]:
# filenamePattern="*_embeddings*.npy"
filenamePattern="*_rawembeddings*.npy"

all_embeddings=[]
for loc in allLocations:
    # find all files
    search_path=loc
    all_embeddings+=list_files(str(search_path),filename=filenamePattern)

print(len(all_embeddings))

20029


In [9]:
# group by loc and time 
# loc: locationID
# time: calendar day

all_embeddings[0:10]

['/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_000000_vgg/S4A10290_20190504_000000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_011602_vgg/S4A10290_20190504_011602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_043000_vgg/S4A10290_20190504_043000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_054602_vgg/S4A10290_20190504_054602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_093000_vgg/S4A10290_20190504_093000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_104602_vgg/S4A10290_20190504_104602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_140000_vgg/S4A10290_20190504_140000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/2019/S4A10290_20190504_151602_vgg/S4A10290_20190504_151602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/46/20

In [28]:
itemSet=set()
regionSet=set()
locationIDset=set()
timeGroupset = set()
all_embeddingsItemDict = {}
for m in all_embeddings:
    splitList = m.split("/")
    region=splitList[6]
    locationID = splitList[7]
    timeGroup = splitList[9].split("_")[1]
    items = (region,locationID,timeGroup)
    itemSet.add(items)
    all_embeddingsItemDict.setdefault(items,[])
    all_embeddingsItemDict[items].append(m)
    regionSet.add(region)
    locationIDset.add(locationID)
    timeGroupset.add(timeGroup)


In [32]:
all_embeddingsItemDict[list(itemSet)[0]]

['/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_003000_vgg/S4A10266_20190512_003000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_014602_vgg/S4A10266_20190512_014602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_053000_vgg/S4A10266_20190512_053000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_064602_vgg/S4A10266_20190512_064602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_100000_vgg/S4A10266_20190512_100000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_111602_vgg/S4A10266_20190512_111602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_150000_vgg/S4A10266_20190512_150000_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/2019/S4A10266_20190512_161602_vgg/S4A10266_20190512_161602_rawembeddings000.npy',
 '/scratch/enis/data/nna/real/anwr/32/20

In [27]:
# regionSet,locationIDset,timeGroupset

In [33]:
# index=0
# for group_embed in all_embeddingsItemDict.values():
#     group_embedLoaded = []
#     for embed_file in (group_embed):
#         embed = np.load(embed_file)
#         group_embedLoaded.append(embed)
#     XX = np.concatenate(group_embedLoaded)
#     XX_normalized = preprocessing.normalize(XX, norm='l2')
#     for ii in range(XX_normalized.shape[0]):
#         t.add_item(index, XX_normalized[ii, :])
#         index += 1
    
#     break
    
# for i, embed_file in enumerate(all_embeddings):
#     embed = np.load(embed_file)
#     for ii in range(embed.shape[0]):
#         t.add_item(index, embed[ii, :])
#         index += 1
#     if i % 1000 == 0:
#         print("Current index:", i)
#     break

In [45]:
# XX = np.concatenate(group_embedLoaded)

(49550, 128)

In [43]:
# X_normalized.shape

(4560, 128)

In [17]:
# sorted(itemSet)

In [29]:
#number of embeddings 11235 for photoEmbeddings_V1

### 2.1) save metadata, embedding file names and sizes as a csv file

In [9]:
embedSizes=[]
for i,embed_file in enumerate(all_embeddings):
    embed=np.load(embed_file)
    embedSizes.append(embed.shape[0])

metadata = list(zip(all_embeddings,embedSizes))
save_to_csv(metadataCSV,metadata)

In [17]:
from annoy import AnnoyIndex

def buildOnDisk(all_embeddings, filePath, vectorLen,distance='euclidean', seed=42, treeN=16):
    filePath = str(filePath)
    t = AnnoyIndex(vectorLen, distance)
    t.on_disk_build(filePath)
    index = 0
    for i, embed_file in enumerate(all_embeddings):
        embed = np.load(embed_file)
        for ii in range(embed.shape[0]):
            t.add_item(index, embed[ii, :])
            index += 1
        if i % 1000 == 0:
            print("Current index:", i)
        # build index
    t.set_seed(seed)
    t.build(treeN)
    return t

In [51]:
10**5

100000

In [52]:
from annoy import AnnoyIndex
from sklearn import preprocessing
def buildOnDiskV2(all_embeddingsItemDict, filePath, vectorLen,distance='euclidean', seed=42, treeN=16):
    filePath = str(filePath)
    t = AnnoyIndex(vectorLen, distance)
    t.on_disk_build(filePath)
    index = 0
    for group_embed in all_embeddingsItemDict.values():
        group_embedLoaded = []
        for embed_file in (group_embed):
            embed = np.load(embed_file)
            group_embedLoaded.append(embed)
        XX = np.concatenate(group_embedLoaded)
        XX_normalized = preprocessing.normalize(XX, norm='l2')
        for ii in range(XX_normalized.shape[0]):
            t.add_item(index, XX_normalized[ii, :])
            index += 1
        if index % 10**5 == 0:
            print("Current index:", i)

    t.set_seed(seed)
    t.build(treeN)
    return t

#### 2.2) Create and save annoy index

In [53]:
p=Path(fn)
fn=str(fn)
if p.exists():
    print(p,"EXISTS")
    t = loadOnDisk(fn,f,distance=distance)
else:
    t = buildOnDiskV2(all_embeddingsItemDict,fn,f,distance=distance,seed=42,treeN=16)

### 3) create functions caches depending on the Index and Metadata

#### 3.1) npy2originalFile

In [45]:
!ls /scratch/enis/data/nna/realMerged/

allEmbeddings_prudhoe26_V11.csv  photoEmbeddings_V1.csv
allEmbeddingsV11.ann		 photoEmbeddings_V2_hamming.ann
allEmbeddings_V11.csv		 photoEmbeddings_V2_hamming.csv
photoEmbeddings_V1.ann		 prudhoe26_V11.ann


In [3]:
import csv
from pathlib import Path
#### LOAD corresponding metadata if different or new

# indexIDname = "photoEmbeddings_V1"
# outputFolder = Path("/scratch/enis/data/nna/realMerged/")
# metadataCSV = outputFolder / (indexIDname+".csv")
# fn = outputFolder / (indexIDname+".ann")

# metadata=[]
# with open(metadataCSV) as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for row in csv_reader:
#         metadata.append((row[0], int(row[1])))

In [ ]:
#NNA folder
nnaProjectDataFolder = Path("/home/enis/projects/nna/data/")

# name and folder of the cached function  file
npy2originalFileCacheFileName = "npy2originalFileCache_"+indexIDname+".pkl"
pklPath = nnaProjectDataFolder / "funcCache" / npy2originalFileCacheFileName


# parameters for the npy2originalFile
# file_properties_df_FilePath = nnaProjectDataFolder / "allFields_dataV3.pkl"
file_properties_df_FilePath = nnaProjectDataFolder / "prudhoeAndAnwr4photoExp_dataV1.pkl"



embeddataPath="/scratch/enis/data/nna/real/"
outputPath=embeddataPath
inputPath="/tank/data/nna/real/"


In [ ]:
import pandas as pd
file_properties_df_FilePath='/home/enis/projects/nna/data/prudhoeAndAnwr4photoExp_dataV1.pkl'
file_properties_df = pd.read_pickle(str(file_properties_df_FilePath))


In [ ]:
from nna.fileUtils import npy2originalfile


In [ ]:
# npy2originalFile

In [ ]:
# parameters ,inputPath,outputPath,file_properties_df

npy2originalFileCache={}
for i,meta in enumerate(metadata):
    orgFilePath = npy2originalfile(Path(meta[0]),outputPath,
                                               file_properties_df,                                               debug=0)
    npy2originalFileCache[meta[0]] = orgFilePath
    if i%1000==0:
        print(i)

In [ ]:
[None,inputPath,outputPath,str(file_properties_df_FilePath),None,None,0]

In [ ]:
npy2originalFileCache["params"] = [None,inputPath,outputPath,str(file_properties_df_FilePath),None,None,0]

In [ ]:
import pickle


with open(pklPath, 'wb') as handle:
    pickle.dump(npy2originalFileCache, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
with open(pklPath, 'rb') as handle:
    b = pickle.load(handle)

print (npy2originalFileCache == b)

In [ ]:
pklPath

### ADD INFO OF SOUND EXISTENCE TO IMG DATATABLE


In [3]:
from search import findFileGetIndex
from utils import loadFiles
from paramsExample import *
import datetime

In [4]:

(file_properties_df, imgOnlyDate, metadata,
NNIndexbyAnnoy, funcCache, dictNumpyTables,
dictDataTable,file_time_locationDict) = loadFiles(file_properties_df_FilePath, imageInfo, metadatCsv,
                                                   annoyIndexPath, vectorSize, distance,
                                                   npy2originalFileCacheFilePath, dictDataTablePath)


In [5]:
handleQueryRequest_params = {
    "file_properties_df":file_properties_df,
    "metadata" : metadata,
    "embeddataPath" : embeddataPath,
    "NNIndexbyAnnoy" : NNIndexbyAnnoy,
    "split_folder" : split_folder,
    "inputPath" : inputPath,
    "outputPath" : outputPath,
    "searchDepth":searchDepth,
    "outputCount":outputCount,
    "queryLen":queryLen,
    "saveFiles":saveFiles,
    "debug":debug,
    "funcCache":funcCache
}

scoresPerImage_params={
    "buffer4ImageMatch":buffer4ImageMatch,
    "file_time_locationDict":file_time_locationDict,
    "dictNumpyTables":dictNumpyTables
}

In [6]:
from nna.fileUtils import find_filesv2

PosixPath('/home/enis/projects/nna/data/prudhoeAndAnwr4photoExp_dataV1.pkl')

In [7]:
def findFileGetIndex(location,start_time,end_time,queryLen,
                     buffer,file_properties_df,metadata,embeddataPath):
    # time-> mp3,flac(original file) -> embed name 000.npy -> embed index
    end_time=start_time+datetime.timedelta(queryLen)

    output = find_filesv2(location,start_time,end_time,queryLen,buffer,file_properties_df)
#     sorted_filtered,start_time,end_time,start_time_org,end_time_org = output
    return output

In [8]:
def handleQueryRequest(location,year,month,day,hour,minute,second,file_properties_df,
                        metadata,embeddataPath,NNIndexbyAnnoy,split_folder,inputPath,
                        outputPath,
                        searchDepth=2,outputCount=5,theta4GroupbyResults=2,
                        queryLen=10,saveFiles=True,debug=0,buffer=0,funcCache=None):
#     #### values from search
#     # /prudhoe/26/2019/S4A10255_20190507_073000_vgg
#     location="26"
#     year=2019;month=5;day=7;hour=7;minute=31;second=10
#     queryLen=10
#     ####
    if funcCache==None:
        funcCache = {}

    start_time=datetime.datetime(year=year,month=month,day=day,
                                 hour=hour,minute=minute,second=second)
    end_time=start_time+datetime.timedelta(queryLen)
    output = findFileGetIndex(location,start_time,end_time,
                                        queryLen,buffer,file_properties_df,
                                        metadata,embeddataPath)
#     embeddingFileIndex=int(embeddingFileIndex)
# #     print(embeddingFileIndex)
#     if embeddingFileIndex<queryLen:
#         if debug>0: print("file not found")
#         return []
    return output

In [9]:
import time

In [79]:
%%time
# def runExp_V1(file_properties_df,dictDataTable,handleQueryRequest_params,scoresPerImage_params):
# allResults={}
c=0
s=time.time()
for loc,dataTable in list(dictDataTable.items())[2:]:
    print(loc)
    audioNotFound=0

#     results4Location={}
#     cc = dataTable[dataTable["Caribou"]>0]
    location=loc
    queryLen=10
    for ID,timestamp in (zip(dataTable["Id"],dataTable["timestamp"])):

        year,month,day,hour,minute,second = (timestamp.year,
                                            timestamp.month,timestamp.day,
                                            timestamp.hour,timestamp.minute,timestamp.second)

        output = handleQueryRequest(location,year,month,day,hour,minute,second,**handleQueryRequest_params)
        output = ([str(i) for i in output[0].index.values]+list(output[1:]))
        if len(output)>4:
            if allResults2[(location,timestamp)] != (output[0],output[1]):
                print((location,timestamp))
                break

        c+=1
        if c%1000==0:
            e=time.time()
            print(c,e-s)
            s=time.time()


#     allResults[loc] = results4Location.copy()



45
1000 32.66358995437622


KeyboardInterrupt: 

In [34]:
len(allResults)

27

In [ ]:
allResults2={}
for loc,IDs in allResults.items():
    if loc=="params":
        continue
    for ID,output in IDs.items():
        
        if len(output)>4:
#             print(loc,ID,output)
            allResults2[(loc,output[-2])] = (output[0],output[1])
        else:
            allResults2[(loc,output[-2])] = None


In [80]:
allResults2["params"] = [None,None,None,10,0,str(file_properties_df_FilePath)]

In [81]:
find_filesv2CacheFolderPath = nnaProjectDataFolder / "funcCache"
# allResultsPath.parent.mkdir()


In [82]:
import pickle
timestr = time.strftime("%Y%m%d-%H%M%S")

find_filesv2CacheFilePath = find_filesv2CacheFolderPath /  ("_".join(["find_filesv2func_cache",timestr])+".pkl")

with open(find_filesv2CacheFilePath, 'wb') as handle:
    pickle.dump(allResults2, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [12]:
# UPDATE paramsExample.py

In [72]:
cc

992711

In [110]:
output=list(allResults2.values())[130:131][0]

In [101]:
(file_properties_df.loc[Path('/tank/data/nna/real/anwr/38/2019/S4A10270_20190504_000000.flac')])

site_id                          38
locationId                       38
site_name                          
recorderId                 S4A10270
hour_min_sec                 000000
year                           2019
month                            05
day                              04
region                         anwr
timestamp       2019-05-04 00:00:00
durationSec                    4560
timestampEnd    2019-05-04 01:16:00
Name: /tank/data/nna/real/anwr/38/2019/S4A10270_20190504_000000.flac, dtype: object

In [98]:
file_properties_df.iloc[0]

site_id                          17
locationId                       17
site_name                          
recorderId                 S4A10307
hour_min_sec                 221602
year                           2019
month                            07
day                              31
region                      prudhoe
timestamp       2019-07-31 22:16:02
durationSec                    4438
timestampEnd    2019-07-31 23:30:00
Name: /tank/data/nna/real/prudhoe/17/2019/S4A10307_20190731_221602.flac, dtype: object

In [112]:
sorted_filteredSeries = file_properties_df.loc[Path(output[0])]

In [113]:
sorted_filteredSeries

site_id                          38
locationId                       38
site_name                          
recorderId                 S4A10270
hour_min_sec                 000000
year                           2019
month                            05
day                              04
region                         anwr
timestamp       2019-05-04 00:00:00
durationSec                    4560
timestampEnd    2019-05-04 01:16:00
Name: /tank/data/nna/real/anwr/38/2019/S4A10270_20190504_000000.flac, dtype: object

In [114]:
sorted_filteredSeries[0]

'38'

In [118]:
file_properties_df.iloc[0].name

PosixPath('/tank/data/nna/real/prudhoe/17/2019/S4A10307_20190731_221602.flac')

In [119]:
sorted_filteredSeries.name

PosixPath('/tank/data/nna/real/anwr/38/2019/S4A10270_20190504_000000.flac')

In [120]:
sorted_filteredSeries.timestamp

Timestamp('2019-05-04 00:00:00')